In [65]:
! pip install ollama openai

In [66]:
import json

messages=[]
messages.append({'role': 'system', 'content': 'You are a helpful customer support assistant.'})

file_path="../tools/tools_register.json"

# Load the JSON data from the file
with open(file_path, 'r') as file:
    tools = json.load(file)
    
    
FUNCTION_MAPPING = dict(
    get_current_weather=lambda city : "sunny"
)

In [67]:
# import ollama

# response = ollama.chat(
#     model='llama3.1',
#     messages=messages,
#     tools=tools,
# )

# print(response['message']['tool_calls'])

# # ollama not support yet
# assistant = client.beta.assistants.create(
#     name="Testing",
#     instructions="Bot for test",
#     tools=tools,
#     model="llama3.1"
# )

In [95]:
def append_to_message(response, messages):
    role, content = response.choices[0].message.role, response.choices[0].message.content
    messages.append(dict(role=role, content=content))


def handle_tool_call(response, messages):
    append_to_message(response, messages)
    
    # Iterate through tool calls to handle each weather check
    for tool_call in response.choices[0].message.tool_calls:
        function = FUNCTION_MAPPING.get(tool_call.function.name)
        arguments = json.loads(tool_call.function.arguments)
        function_output = function(**arguments)
        
        tool_output = {
                "role": "tool",
                "tool_call_id": tool_call.id
            }
        
        
        if tool_call.function.name == "some_specific_name":
            # need to engineer the output content
            pass
        elif tool_call.function.name == "get_current_weather":
            tool_output["content"] = json.dumps(dict(weather=function_output))
        else:
            tool_output["content"] = json.dumps(function_output)
        
        
        messages.append(tool_output)

def handle_normal_response(response, messages):
    append_to_message(response, messages)
    print("Assistant: " + response.choices[0].message.content)
    

In [96]:
import openai
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

our_api_request_forced_a_tool_call = False

In [97]:
messages.append({'role': 'user', 'content': 'try it again'})


In [98]:

while(True):    
    response = client.chat.completions.create(
        model="llama3.1",
        messages=messages,
        tools=tools,
    )
    
    # Check if the conversation was too long for the context window
    if response.choices[0].finish_reason == "length":
        print("Error: The conversation was too long for the context window.")
        # Handle the error as needed, e.g., by truncating the conversation or asking for clarification
        handle_length_error(response)
        break
        
    # Check if the model's output included copyright material (or similar)
    if response.choices[0].finish_reason == "content_filter":
        print("Error: The content was filtered due to policy violations.")
        # Handle the error as needed, e.g., by modifying the request or notifying the user
        handle_content_filter_error(response)
        break
        
    # Check if the model has made a tool_call. This is the case either if the "finish_reason" is "tool_calls" or if the "finish_reason" is "stop" and our API request had forced a function call
    if (response.choices[0].finish_reason == "tool_calls" or 
        # This handles the edge case where if we forced the model to call one of our functions, the finish_reason will actually be "stop" instead of "tool_calls"
        (our_api_request_forced_a_tool_call and response.choices[0].finish_reason == "stop")):
        # Handle tool call
        print("Log: Model made a tool call.")
        # Your code to handle tool calls
        handle_tool_call(response, messages)
        continue
        
    # Else finish_reason is "stop", in which case the model was just responding directly to the user
    elif response.choices[0].finish_reason == "stop":
        # Handle the normal stop case
        print("Log: Model responded directly to the user.")
        # Your code to handle normal responses
        handle_normal_response(response, messages)
        break
        
    # Catch any other case, this is unexpected
    else:
        print("Unexpected finish_reason:", response.choices[0].finish_reason)
        # Handle unexpected cases as needed
        handle_unexpected_case(response)
        break

Log: Model made a tool call.
Log: Model responded directly to the user.
Assistant: Ha ha, looks like you're stuck in sunny mode!

Seriously though, I can try to simulate different scenarios for you. Let's say... what if we updated the output to {"weather": "cloudy"}?


In [99]:
messages

[{'role': 'system',
  'content': 'You are a helpful customer support assistant.'},
 {'role': 'user', 'content': 'try it again'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9225di2c', function=Function(arguments='{"city":"New York City"}', name='get_current_weather'), type='function')]),
 {'role': 'user', 'content': 'try it again'},
 ChatCompletionMessage(content='You want me to try the tool call response with another input.\n\nSince I don\'t have any new information, we can use the same data:\n \n{"type":"function","name":null,"parameters":{}}\n\n\n\nSince there\'s no function call output, please provide the one you\'d like to test.', refusal=None, role='assistant', function_call=None, tool_calls=None),
 ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=None),
 {'role': 'user', 'content': 'try it again'},
 {'role': 'assistant', 'content': ''},
 {